In [1]:
import pandas as pd
from dateutil import parser

In [2]:
events = pd.read_csv('Truck Events Files/operator event ver 1.csv',encoding='cp1252')
events_severe = events[events['EVNT_LVL'] != 1.0]
events_severe = events_severe.dropna(how='all')

In [3]:
def clean_event_date_time(event):
    real_time = event.OCUR_TS[0:8]
    pm_or_am = event.OCUR_TS[-2:]
    
    cleaned_time = real_time + pm_or_am
    cleaned_time = cleaned_time.replace(".", ":")
    
    cleaned_date = event.OCUR_TS_Date
    
    cleaned_date_time = cleaned_date + " " + cleaned_time

    return parser.parse(cleaned_date_time)

In [4]:
events_severe['timestamp'] = events_severe.apply(clean_event_date_time, axis=1)
truck_events = {}
for truck in events_severe['MACH_SER_NO'].unique():
    truck_events.update({truck: events_severe[events_severe['MACH_SER_NO'] == truck]})

In [5]:
eventfunctions = {}

In [6]:
def return_self(val):
    return val

In [7]:
def alpha(event):
    a = event['DATA_VAL']
    b = event['ACT_LMT']
    if b != 0:
        return abs((a-b)/b)
    return 1

In [11]:
def event_instance_severity(event):
    event_id = event['EVNT_ID']
    event_function = eventfunctions.get(event_id, return_self)
    try:
        event_duration = float(event['DUR'])
    except ValueError:
        return 0
    return event_function(alpha(event))*event_duration

In [12]:
events_severe_test = events_severe
events_severe_test['severity'] = events_severe.apply(event_instance_severity, axis=1)

In [13]:
events_severe_test.head(10)

,MACH_SER_NO,PRIM_INFO_TYP,SCND_INFO_TYP,SCND_INFO_DESC,PRIM_CMPNT_DESC,SCND_CMPNT_DESC,OCUR_TS_Date,OCUR_TS,SMU,MID,...,DUR,ACT_LMT,EVNT_ID,CID,ACK_TM,ACK_NO,OCUR_TS_L,OCAL,timestamp,severity
0,Truck 6,EVENT,Wastegate Solenoid,NaN,ENGINE,AIR INDUCTION & EXHAUST SYSTEM,7-Oct-15,01.25.56.000000000 AM,23271.55417,81.0,...,159,0.0,766.0,766.0,0.0,1.0,6-Oct-15,08.25.56.000000000 PM,2015-10-07 01:25:56,159.000000
1,Truck 6,EVENT,Wastegate Solenoid,NaN,ENGINE,AIR INDUCTION & EXHAUST SYSTEM,7-Oct-15,01.41.19.000000000 AM,23271.81056,81.0,...,7,0.0,766.0,766.0,0.0,0.0,6-Oct-15,08.41.19.000000000 PM,2015-10-07 01:41:19,7.000000
2,Truck 6,EVENT,Wastegate Solenoid,NaN,ENGINE,AIR INDUCTION & EXHAUST SYSTEM,7-Oct-15,01.43.46.000000000 AM,23271.85139,81.0,...,5,0.0,766.0,766.0,0.0,0.0,6-Oct-15,08.43.46.000000000 PM,2015-10-07 01:43:46,5.000000
3,Truck 6,EVENT,Wastegate Solenoid,NaN,ENGINE,AIR INDUCTION & EXHAUST SYSTEM,7-Oct-15,01.50.21.000000000 AM,23271.96111,81.0,...,5,0.0,766.0,766.0,0.0,0.0,6-Oct-15,08.50.21.000000000 PM,2015-10-07 01:50:21,5.000000
4,Truck 6,EVENT,Wastegate Solenoid,NaN,ENGINE,AIR INDUCTION & EXHAUST SYSTEM,7-Oct-15,01.53.14.000000000 AM,23272.00917,81.0,...,32,0.0,766.0,766.0,0.0,1.0,6-Oct-15,08.53.14.000000000 PM,2015-10-07 01:53:14,32.000000
5,Truck 6,EVENT,Wastegate Solenoid,NaN,ENGINE,AIR INDUCTION & EXHAUST SYSTEM,7-Oct-15,01.55.47.000000000 AM,23272.05167,81.0,...,8,0.0,766.0,766.0,0.0,0.0,6-Oct-15,08.55.47.000000000 PM,2015-10-07 01:55:47,8.000000
6,Truck 6,EVENT,Wastegate Solenoid,NaN,ENGINE,AIR INDUCTION & EXHAUST SYSTEM,7-Oct-15,12.58.31.000000000 PM,23274.66333,81.0,...,78,0.0,766.0,766.0,38.0,1.0,7-Oct-15,07.58.31.000000000 AM,2015-10-07 12:58:31,78.000000
7,Truck 1,EVENT,Turbo Inlet Temperature Left High,NaN,DRIVE TRAIN,FINAL DRIVE,6-Oct-15,05.33.13.000000000 PM,21634.74750,36.0,...,ACTIVE,729.0,246.0,246.0,6.0,2.0,6-Oct-15,12.33.13.000000000 PM,2015-10-06 17:33:13,0.000000
8,Truck 10,EVENT,Turbo Inlet Temperature Left High,NaN,DRIVE TRAIN,FINAL DRIVE,13-Jul-16,12.08.44.000000000 AM,22441.67444,36.0,...,1024,764.0,246.0,246.0,0.0,1.0,12-Jul-16,07.08.44.000000000 PM,2016-07-13 00:08:44,12.062827
9,Truck 10,EVENT,Turbo Inlet Temperature Left High,NaN,DRIVE TRAIN,FINAL DRIVE,13-Jul-16,01.02.22.000000000 AM,22442.56833,36.0,...,1483,747.0,246.0,246.0,0.0,1.0,12-Jul-16,08.02.22.000000000 PM,2016-07-13 01:02:22,19.852744
